In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter,AutoMinorLocator
import matplotlib.patches as patches

import pickle
from tqdm import tqdm
import pandas as pd

f = open('/Users/admin/Desktop/data_bowl/full_plays_dictionary.pckl', 'rb')
dat = pickle.load(f)
f.close()

#Get player info
df_p = pd.read_csv('data_bowl_github/raw_data/players.csv')

#Load in the EPT for this play
f = open('/Users/admin/Desktop/data_bowl/EPT_variables_2775.pkl', 'rb')
time_plot,ept_plot = pickle.load(f)
f.close()


In [89]:
def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='white', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='k')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='k', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='k')
        ax.plot([x, x], [53.0, 52.5], color='k')
        ax.plot([x, x], [22.91, 23.57], color='k')
        ax.plot([x, x], [29.73, 30.39], color='k')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax


In [241]:
frame = '2021091913-2775'

#Get each frame
game_dat = dat['pass'][frame]

#plot each frame here
frames = list(dat['pass'][frame].keys())
for j in range(6,55):

    #Get the variables needed to make the plot--------------    

    #Create the figure
    #fig,ax = create_football_field(figsize=(8,8))
    fig = plt.figure(figsize=(12,7))
    plt.grid(color='k',linewidth=2)
    plt.gca().yaxis.grid(False)
    ax = plt.subplot(1,1,1)
    #plt.title('{}: Frame {}'.format(frame,frames[j]),loc='left')
    
    
    #Add the EPT graph 
    plt.rcParams.update({'font.size':16,'xtick.top':False,'xtick.labeltop':False,'ytick.left':True,'xtick.bottom':True})
    ax2 = fig.add_axes([0.026, 0.701, 0.3, 0.2], anchor='NE', zorder=1) #0.58
    ax2.grid(alpha=0.6)
    ax2.plot(time_plot[:j-6],ept_plot[:j-6],color='blue')
    
    #Format the axes
    #ax2.set_xlabel('Play Duration [s]')
    ax2.xaxis.set_major_locator(MultipleLocator(1))
    ax2.xaxis.set_major_formatter(FormatStrFormatter('%d'))
    ax2.xaxis.set_minor_locator(MultipleLocator(0.5))
    ax2.set_xlim([0,5])
    
    #ax2.set_ylabel('Pocket Time Left [s]')
    ax2.yaxis.set_major_locator(MultipleLocator(1))
    ax2.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax2.yaxis.set_minor_locator(MultipleLocator(0.5))
    ax2.set_ylim([0,4])
    
    plt.rcParams.update({'font.size':20,'xtick.top':True,'xtick.labeltop':True,'ytick.left':False})
    
    #Get the playerIDs and positions currently on the field for this play
    playerIDs = list(dat['pass'][frame][str(frames[j])].keys())
    playerIDs.remove('-999.0')
    player_pos = np.array([df_p.loc[np.where(df_p.nflId==int(float(val)))[0][0]]['officialPosition'] for val in playerIDs])
    playerIDs.append('-999.0')
    
    play_dat = dat['pass'][frame][str(frames[j])]
    
    #Go through each player ID and plot their stuff
    all_x,all_y = [],[]
    red,blue = False,False
    for k in range(len(playerIDs)):
        #Get the home and away team color coded
        full_dat = play_dat[str(playerIDs[k])]
        if playerIDs[k] == '-999.0':
            playerIDs[k] = -999
        if full_dat[-1]=='football':
            color = 'k'
        elif full_dat[-1]=='home':
            color = 'red'
        elif full_dat[-1]=='away':
            color = 'blue'

        #Plot x,y coordinates of lineman only, football, and QB
        good_pos = ['T','G','C','DT','DE','NT','QB']
        
        if (float(playerIDs[k])==-999.) or (player_pos[k] in good_pos):
            
            xpos,ypos = float(full_dat[0]),float(full_dat[1])
            
            if (float(playerIDs[k])!=-999.):
                if color=='red':
                    if red == False:
                        ax.scatter(xpos,ypos,s=800,color=color,edgecolors='k',label='Offense')
                        red = True
                    else:
                        ax.scatter(xpos,ypos,s=800,color=color,edgecolors='k')
                elif color=='blue':
                    if blue == False:
                        ax.scatter(xpos,ypos,s=800,color=color,edgecolors='k',label='Defense')
                        blue = True
                    else:
                        ax.scatter(xpos,ypos,s=800,color=color,edgecolors='k')
                    
            else:
                ax.scatter(xpos,ypos,s=300,color=color,edgecolors='k',label='Football')

            #Plot the vector of the motion of the football
            s,o = float(full_dat[2]),float(full_dat[6])
            u,v = s*np.cos(np.radians(90-o)),s*np.sin(np.radians(90-o))
            #ax.quiver(float(full_dat[0]),float(full_dat[1]),u,v,color=color)

            #Add the label of the position
            if (float(playerIDs[k])!=-999.):
                ax.annotate(str(int(float(full_dat[7]))),xy=(xpos,ypos),xytext=(0, 0),textcoords='offset points', ha='center',fontsize=18,color='white',va='center')

            #Format the axes
            ax.xaxis.set_major_locator(MultipleLocator(5))
            ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
            ax.xaxis.set_minor_locator(MultipleLocator(1))
            
            
            #Append all values to make the bounds of the plot
            all_x.append(xpos)
            all_y.append(ypos)
    
    ax.set_xlim([np.min(np.array(all_x))-5,np.max(np.array(all_x))+5])
    ax.set_ylim([np.min(np.array(all_y))-5,np.max(np.array(all_y))+5])
    
    #Make the legend
    lgnd = ax.axes.legend(loc='upper right',framealpha=0,fontsize=18)
    lgnd.legendHandles[0]._sizes[0] = 200
    lgnd.legendHandles[1]._sizes[0] = 200
    lgnd.legendHandles[2]._sizes[0] = 200
    
    #Remove y axis tick marks (like a football field)
    ax.axes.tick_params(left = False, right = False , labelleft = False)
    
    plt.tight_layout()
    plt.savefig('play_vids/{}_{}.jpg'.format(frame,frames[j]),dpi=300,bbox_inches='tight')
    #plt.show()
    plt.close()

    #break
    
    

/var/folders/r2/d8hpfhkd33j9w42_6twnlsym0000gn/T/ipykernel_17301/2028548668.py:120: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
